# <center> Tâche 1 de la pipeline : transcription des audios à l'écrit</center>

A partir de nos données audios (générées dans le notebook *Dataset_Creation*), nous allons les transcrire à l'écrit en utilisant un modèle de *speech to text*. Le **défi** ici est de parvenir à transcrire les audios en conservant les erreurs afin de permettre à l'utilisateur de visualiser ses erreurs dans le texte, et par la suite le texte corrigé. Le problème est que la plupart des modèles de *speech to text* possèdent une couche de traitement de la langue leur permettant de corriger les fautes. Un **autre défi** auquel nous allons faire face est l'affinnage du modèle, pour tenter d'augmenter ses performances en particulier sur l'accent indien.

On commence par créer notre jeu de données comportant les audios précédents et leur transcriptions associées. Le but ici est de créer un jeu de données qui servira pour l'affinnage de notre STT.

In [1]:
import pandas as pd
import os
import torchaudio
import torch
from datasets import load_dataset, concatenate_datasets, Dataset
from datasets.features.audio import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from gtts import gTTS
import import_ipynb
from Dataset_Creation import dataset_concat, nb_audio, audio_folder # import de variables provenant de l'autre notebook

importing Jupyter notebook from Dataset_Creation.ipynb
Les retranscriptions audios se trouvent dans le dossier 'audio'. 
Nombre total d'audio créé: 1141 sur 1501 textes du jeu de données JFLEG


In [2]:
# CREATION METADATA (doc hugging face :https://huggingface.co/docs/datasets/audio_load)

# Liste des chemins des fichiers audio
audio_files = [f"{i}.mp3" for i in range(0, nb_audio)]

# Transcriptions associées
transcriptions = dataset_concat["sentence"][:nb_audio]

# Créer un DataFrame avec les noms de fichiers et les transcriptions
df = pd.DataFrame({"file_name": audio_files, "transcription": transcriptions})

# Enregistrer le DataFrame dans un fichier CSV
metadata_csv_path = os.path.join(audio_folder, "metadata.csv")
df.to_csv(metadata_csv_path, index=False)

print(f"Metadata CSV enregistré à : {metadata_csv_path}")

# CREATION DU LOAD_DATASET
dataset = load_dataset("audiofolder", data_dir=audio_folder, split="train")

Metadata CSV enregistré à : ./audio\metadata.csv


Resolving data files:   0%|          | 0/1142 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 1141
})

In [4]:
dataset['audio']

[{'path': 'C:/Users/beaus/OneDrive - Université Laval/Automne 2023/MLSP/Projet/Repo/MLSP_Project/audio/0.mp3',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -6.14001419e-06, -6.29437864e-06, -6.43266594e-06]),
  'sampling_rate': 16000},
 {'path': 'C:/Users/beaus/OneDrive - Université Laval/Automne 2023/MLSP/Projet/Repo/MLSP_Project/audio/1.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 16000},
 {'path': 'C:/Users/beaus/OneDrive - Université Laval/Automne 2023/MLSP/Projet/Repo/MLSP_Project/audio/10.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 16000},
 {'path': 'C:/Users/beaus/OneDrive - Université Laval/Automne 2023/MLSP/Projet/Repo/MLSP_Project/audio/100.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 16000},
 {'path': 'C:/Users/beaus/OneDrive - Université Laval/Automne 2023/MLSP/Projet/Repo/MLSP_Project/audio/1000.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),

Nos données avaient initialement un taux d'échantillonnage (*sampling_rate*) de 24KHz, or le modèle STT d'Hugging face a besoin d'un taux de 16KHz (doc : https://huggingface.co/facebook/wav2vec2-base-960h). On a donc fait un rééchantillonnage.

In [8]:
# # Liste des chemins des fichiers audio
# audio_files = [os.path.join(audio_folder, f"{i}.mp3") for i in range(0, nb_audio)]

# # Rééchantillonnage des fichiers audio à 16000 Hz
# target_sampling_rate = 16000
# resample_transform = torchaudio.transforms.Resample(orig_freq=24000, new_freq=target_sampling_rate)

# # Rééchantillonner et sauvegarder chaque fichier audio
# for audio_file in audio_files:
#     # Charger l'audio
#     waveform, original_sampling_rate = torchaudio.load(audio_file, normalize=True)

#     # Rééchantillonner l'audio
#     resampled_waveform = resample_transform(waveform)

#     # Sauvegarder l'audio rééchantillonné dans un nouveau fichier
#     new_audio_file = os.path.join(audio_folder, os.path.basename(audio_file))
#     torchaudio.save(new_audio_file, resampled_waveform, target_sampling_rate)

#     # print(f"Audio rééchantillonné enregistré : {new_audio_file}")


In [5]:
dataset["audio"][0]

{'path': 'C:/Users/beaus/OneDrive - Université Laval/Automne 2023/MLSP/Projet/Repo/MLSP_Project/audio/0.mp3',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -6.14001419e-06, -6.29437864e-06, -6.43266594e-06]),
 'sampling_rate': 16000}

Nos données sont maintenant bien échantillonées à 16KHz.

In [7]:
from IPython.display import Audio

sample = dataset[2] # échantillon pour tester le modèle

print(sample["transcription"])
Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

Also the work of the media , television , magazines , and every stuff made for publicity affects the emotions of many people around the world , encouraging them to buy and buy anything they can . 
